In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp cli

# CLI 
> Contains functions for calling Rough from the command line

In [ ]:
#| export
from fastcore.script import *
import numpy as np
from pathlib import Path
from inspect import getmembers, isfunction

from rough.data import *
from rough.profile import *
from rough.section import *

import rough.profile as profile_mod
import rough.section as section_mod

In [ ]:
array = np.loadtxt('example.txt')
array.shape

(501, 501)

In [ ]:
#| export
def compute_parameters(array, #Input array to be calculate paramers on
                       list_of_parameters:list, #List of parameters to calculate as strings
                       valid_module  = None, #module to generate functions from, used to check user input, see rough.cli:rough
                       **kwargs #Keyword arguments to modify behavior of parameter calls, usually to define Sections = True
                      ):
    
    results = []
    
    #The following generates a {'func':func} dict from given list of parameters if the parameter is available in the module
    valid_dict = {k: v for k, v in vars(valid_module).items() if callable(v) and k in valid_module.__all__}
    for parameter in list_of_parameters:
        result = valid_dict[parameter](array, kwargs)
        results.append(result)
    return results

In [ ]:
#| export
@call_parse
def rough(
    fname:str   = None,   #File name, path or directory with data files to be read
    ext:str     = '.txt', #Extension for the  files .txt or .csv
    
    result:str     = None,     #Directory to write results to, if None, writes to 'results'
    result_how:str = 'concat', #How to save the results, 'concat' concatenates all respective types of results 
                               #(i.e. profile,section,rotational,subsection) into one dataframe file.
                               #'split' produces respective result files for each input file. Use split for large amounts of data.
    
    level:bool  = True, #Perform plane levelling 
    form:bool   = True, #Remove form by polynomial subtraction
    deg:int     = 3,    #Degree of polynomial to remove
    smooth:bool = True, #Smooth the array by applying a gaussian
    sigma:int   = 1,    #Sigma for gaussian to be applied
    
    gen_rot:bool= True, #Generate rotational profiles and apply parameter calculation to them
    
    gen_section:bool= True, #Generate sub-sections of the surface
    sec_how:str = 'square', #Type of section to generate, currently only supports 'square'
    sec_num:int = 100,      #Number of sections to generate
    
    profile:bool = True, #Calculate profile parameters
    section:bool = True, #Calculate section parameters
    
    params1D:list = profile_mod.__all__, # list of 1D parameters to calculate,
    params2D:list = section_mod.__all__, #list of 2D parameters to calculate, calculates for both the sections and the whole
):
    
    delims = {'.txt': None,
              '.csv': ','}
    
    path       = Path().cwd() if fname == None else Path(fname)
    
    #Figuring out where the results go
    if   path.is_dir():
        result_dir = path / 'results' if result == None else Path(result)
    elif path.is_file():
        result_dir = path.parent / 'results' if result == None else Path(result) 
    

    if not path.exists(): 
        raise FileNotFoundError('Could not find file/directory check fname')
    if not result_dir.exists(): 
        result_dir.mkdir(parents=True) #Make the results directory if it doesn't exist
        
    glob_pattern    = '*' + ext
    file_paths = [path] if path.is_file() else path.glob(glob_pattern)
    
    for file_path in file_paths:
        array = np.loadtxt(file_path,delimiter=delims[file_path.suffix])
        print(file_path)
        
        if level:
            array = plane_level(array)
            print('Got to level')
        if form:
            array = remove_form(array)
            print('Got to form')
        if smooth:
            array = smooth_image(array,sigma=sigma)
            print('got to smooth')
        
        if profile: #Iterate through the profile parameters
            profile_results = compute_parameters(array, params1D, profile_mod)
            if gen_rot:
                profiles = gen_rot_prof(array)
                rot_profile_results = compute_parameters(profiles, params1D, profile_mod)
        
        if section: #Iterate through profile parameters
            section_results = compute_parameters(array, params2D, section_mod)
            
            if gen_section:
                sections         = gen_sections(array, how=sec_how, number = sec_num)
                sections_results = compute_parameters(sections, params2D, section_mod, sections = True)

                result_file_name = file_path.stem + '_section' + file_path.suffix #TODO add in .csv and other filer support
                result_path = result_dir / result_file_name 
                print(result_path)
            #np.savetxt(result_path, section_results, delim =delim) #Save the section results

In [ ]:
path = Path()/'nonexistent_file.txt'
pattern = '*.txt'
path.exists()

False

In [ ]:
path = Path.cwd()
path

Path('E:/Archaeology/rough/rough')

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()